# Ejercicio 2: HIVE.
Ejemplo de uso de HIVE sobre 2 sets de datos.

In [1]:
! mkdir -p /media/notebook/tp1-notebooks/ejer_2_hive

In [2]:
import os
os.chdir("/media/notebook/tp1-notebooks/ejer_2_hive")

In [3]:
! pwd

/media/notebook/tp1-notebooks/ejer_2_hive


In [4]:
! hadoop fs -mkdir -p /user/uned/databases/ejer2_hive

In [5]:
! hadoop fs -ls /user/uned/databases/

Found 1 items
drwxr-xr-x   - root supergroup          0 2019-11-06 23:56 /user/uned/databases/ejer2_hive


In [6]:
! hadoop fs -mkdir -p /user/uned/databases/ejer2_hive/external/TablaExternaGDP
! hadoop fs -mkdir -p /user/uned/databases/ejer2_hive/external/TablaExternaEscolarizacion

In [7]:
! hadoop fs -ls /user/uned/databases/ejer2_hive/external/

Found 2 items
drwxr-xr-x   - anonymous supergroup          0 2019-11-06 23:54 /user/uned/databases/ejer2_hive/external/TablaExternaEscolarizacion
drwxr-xr-x   - root      supergroup          0 2019-11-06 23:55 /user/uned/databases/ejer2_hive/external/TablaExternaGDP


In [8]:
%%writefile create_tables.hql

--Creamos la BD que contendra las tablas
CREATE DATABASE IF NOT EXISTS ejer2_hive
COMMENT 'BD para el ejercicio 2 del TP1'
LOCATION '/user/uned/databases/ejer2_hive';

--Borramos las tablas si existen.
DROP TABLE IF EXISTS ejer2_hive.TablaInternaGDP;
DROP TABLE IF EXISTS ejer2_hive.TablaExternaGDP;
DROP TABLE IF EXISTS ejer2_hive.TablaInternaEscolarizacion;
DROP TABLE IF EXISTS ejer2_hive.TablaExternaEscolarizacion;

--Creamos TablaExternaGDP en la BD ejer2_hive
CREATE EXTERNAL TABLE ejer2_hive.TablaExternaGDP ( Country_Name STRING,
Country_Code STRING,
Indicator_Name STRING,
Indicator_Code STRING,
`1960` STRING,`1961` STRING,`1962` STRING,`1963` STRING,
`1964` STRING,`1965` STRING,`1966` STRING,`1967` STRING,
`1968` STRING,`1969` STRING,`1970` STRING,`1971` STRING,
`1972` STRING,`1973` STRING,`1974` STRING,`1975` STRING,
`1976` STRING,`1977` STRING,`1978` STRING,`1979` STRING,
`1980` STRING,`1981` STRING,`1982` STRING,`1983` STRING,
`1984` STRING,`1985` STRING,`1986` STRING,`1987` STRING,
`1988` STRING,`1989` STRING,`1990` STRING,`1991` STRING,
`1992` STRING,`1993` STRING,`1994` STRING,`1995` STRING,
`1996` STRING,`1997` STRING,`1998` STRING,`1999` STRING,
`2000` STRING,`2001` STRING,`2002` STRING,`2003` STRING,
`2004` STRING,`2005` STRING,`2006` STRING,`2007` STRING,
`2008` STRING,`2009` STRING,`2010` STRING,`2011` STRING,
`2012` STRING,`2013` STRING,`2014` STRING,`2015` STRING,
`2016` STRING
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ","
LINES TERMINATED BY "\n"
STORED AS TEXTFILE
LOCATION "/user/uned/databases/ejer2_hive/external/TablaExternaGDP";

--Creamos TablaInternaGDP en la BD ejer2_hive
CREATE TABLE ejer2_hive.TablaInternaGDP AS SELECT * FROM ejer2_hive.TablaExternaGDP;

--Creamos TablaExternaEscolarizacion en la BD ejer2_hive
CREATE EXTERNAL TABLE ejer2_hive.TablaExternaEscolarizacion ( Country_Name STRING,
Country_Code STRING,
Indicator_Name STRING,
Indicator_Code STRING,
`1960` STRING,`1961` STRING,`1962` STRING,`1963` STRING,
`1964` STRING,`1965` STRING,`1966` STRING,`1967` STRING,
`1968` STRING,`1969` STRING,`1970` STRING,`1971` STRING,
`1972` STRING,`1973` STRING,`1974` STRING,`1975` STRING,
`1976` STRING,`1977` STRING,`1978` STRING,`1979` STRING,
`1980` STRING,`1981` STRING,`1982` STRING,`1983` STRING,
`1984` STRING,`1985` STRING,`1986` STRING,`1987` STRING,
`1988` STRING,`1989` STRING,`1990` STRING,`1991` STRING,
`1992` STRING,`1993` STRING,`1994` STRING,`1995` STRING,
`1996` STRING,`1997` STRING,`1998` STRING,`1999` STRING,
`2000` STRING,`2001` STRING,`2002` STRING,`2003` STRING,
`2004` STRING,`2005` STRING,`2006` STRING,`2007` STRING,
`2008` STRING,`2009` STRING,`2010` STRING,`2011` STRING,
`2012` STRING,`2013` STRING,`2014` STRING,`2015` STRING,
`2016` STRING
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ","
LINES TERMINATED BY "\n"
STORED AS TEXTFILE
LOCATION "/user/uned/databases/ejer2_hive/external/TablaExternaEscolarizacion";

--Creamos TablaInternaEscolarizacion en la BD ejer2_hive
CREATE TABLE ejer2_hive.TablaInternaEscolarizacion AS SELECT * FROM ejer2_hive.TablaExternaEscolarizacion;


Overwriting create_tables.hql


In [9]:
! beeline -u "jdbc:hive2://localhost:10000/default" -f create_tables.hql

2019-11-06 23:58:50,357 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/default> 
s: jdbc:hive2://localhost:10000/default> --Creamos la BD que contendra las tabla 
e: jdbc:hive2://localhost:10000/default> CREATE DATABASE IF NOT EXISTS ejer2_hiv 
': jdbc:hive2://localhost:10000/default> COMMENT 'BD para el ejercicio 2 del TP1 
ve';dbc:hive2://localhost:10000/default> LOCATION '/user/uned/databases/ejer2_hi 
INFO  : Compiling command(queryId=hive_20191106235858_deba9d25-ba82-4ebd-9798-6cceba6c7db9): CREATE DATABASE IF NOT EXISTS ejer2_hive
COMMENT 'BD para el ejercicio 2 del TP1'
LOCATION '/user/uned/databases/ejer2_hive'
INFO  : Semantic Analysis Co

In [10]:
#Copiamos los datos a HDFS...
! hadoop fs -put /media/notebook/datos/API_NY.GDP.MKTP.CD_DS2_en_csv_v2.csv \
/user/uned/databases/ejer2_hive/external/TablaExternaGDP
! hadoop fs -ls /user/uned/databases/ejer2_hive/external/TablaExternaGDP

put: `/user/uned/databases/ejer2_hive/external/TablaExternaGDP/API_NY.GDP.MKTP.CD_DS2_en_csv_v2.csv': File exists
Found 1 items
-rw-r--r--   1 root supergroup     205586 2019-11-06 23:55 /user/uned/databases/ejer2_hive/external/TablaExternaGDP/API_NY.GDP.MKTP.CD_DS2_en_csv_v2.csv


In [11]:
#Copiamos los datos a HDFS...
! hadoop fs -put /media/notebook/datos/API_SE.PRM.UNER.FE_DS2_en_csv_v2.csv \
/user/uned/databases/ejer2_hive/external/TablaExternaEscolarizacion
! hadoop fs -ls /user/uned/databases/ejer2_hive/external/TablaExternaEscolarizacion

put: `/user/uned/databases/ejer2_hive/external/TablaExternaEscolarizacion/API_SE.PRM.UNER.FE_DS2_en_csv_v2.csv': File exists
Found 1 items
-rw-r--r--   1 root supergroup      63371 2019-11-06 23:54 /user/uned/databases/ejer2_hive/external/TablaExternaEscolarizacion/API_SE.PRM.UNER.FE_DS2_en_csv_v2.csv


In [12]:
# Comprobamos que tienen datos:
! beeline -u "jdbc:hive2://localhost:10000/default" -e "SELECT * FROM ejer2_hive.TablaExternaGDP limit 5"

2019-11-06 23:59:47,242 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20191106235959_d720976b-4a2e-48c7-a868-aea1d850089d): SELECT * FROM ejer2_hive.TablaExternaGDP limit 5
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:tablaexternagdp.country_name, type:string, comment:null), FieldSchema(name:tablaexternagdp.country_code, type:string, comment:null), FieldSchema(name:tablaexternagdp.indicator_name, type:string, comment:null), FieldSchema(name:tablaexternagdp.indicator_code, type:string, comment:null), FieldSchema(name:tablaexternagdp.1960, type:string, comment:null), FieldSchema(nam

In [13]:
# Comprobamos que tienen datos:
! beeline -u "jdbc:hive2://localhost:10000/default" -e "SELECT * FROM ejer2_hive.TablaInternaGDP limit 5"

2019-11-06 23:59:51,450 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20191106235959_be1788e8-cf21-4c92-9a95-3b7be4092858): SELECT * FROM ejer2_hive.TablaInternaGDP limit 5
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:tablainternagdp.country_name, type:string, comment:null), FieldSchema(name:tablainternagdp.country_code, type:string, comment:null), FieldSchema(name:tablainternagdp.indicator_name, type:string, comment:null), FieldSchema(name:tablainternagdp.indicator_code, type:string, comment:null), FieldSchema(name:tablainternagdp.1960, type:string, comment:null), FieldSchema(nam

In [14]:
# Comprobamos que tienen datos:
! beeline -u "jdbc:hive2://localhost:10000/default" -e "SELECT * FROM ejer2_hive.TablaExternaEscolarizacion limit 5"

2019-11-07 00:00:03,446 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20191107000000_8bc36680-6f80-4c8c-88f8-61b76488eec1): SELECT * FROM ejer2_hive.TablaExternaEscolarizacion limit 5
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:tablaexternaescolarizacion.country_name, type:string, comment:null), FieldSchema(name:tablaexternaescolarizacion.country_code, type:string, comment:null), FieldSchema(name:tablaexternaescolarizacion.indicator_name, type:string, comment:null), FieldSchema(name:tablaexternaescolarizacion.indicator_code, type:string, comment:null), FieldSchema(name:tablaexter

In [15]:
# Comprobamos que tienen datos:
! beeline -u "jdbc:hive2://localhost:10000/default" -e "SELECT * FROM ejer2_hive.TablaInternaEscolarizacion limit 5"

2019-11-07 00:00:18,978 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20191107000000_1538f632-1a2a-4ac3-a3c5-ca5e3f89bb9c): SELECT * FROM ejer2_hive.TablaInternaEscolarizacion limit 5
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:tablainternaescolarizacion.country_name, type:string, comment:null), FieldSchema(name:tablainternaescolarizacion.country_code, type:string, comment:null), FieldSchema(name:tablainternaescolarizacion.indicator_name, type:string, comment:null), FieldSchema(name:tablainternaescolarizacion.indicator_code, type:string, comment:null), FieldSchema(name:tablainter